# Used Code for the Thesis:

# Dark Energy: Constraining Cosmological Models with Extra Spatial Dimensions Based on Type Ia Supernovae Data

## $\Lambda$CDM Model

### Fitting the Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Funktion zum Laden der Pantheon+SH0ES-Daten
def load_pantheon_sh0es_data(data_file, cov_file):
    """
    Lädt die Pantheon+SH0ES-Daten und die Kovarianzmatrix.
    """
    # Laden der Daten aus der .dat-Datei
    dat = pd.read_csv(data_file, sep=' ')
    
    # Extrahiere relevante Spalten
    z = np.array(dat['zHD'])  # Redshifts
    dm = np.array(dat['MU_SH0ES'])  # Distanzmoduli
    
    # Laden der Kovarianzmatrix aus der .cov-Datei
    ncol = 1701  # Anzahl der Spalten in der Kovarianzmatrix (Pantheon+SH0ES)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)
    
    # Auswahlkriterien für die Supernovae (z > 0.01)
    selnotlocal = z > 0.01
    select = selnotlocal
    
    # Anwenden der Auswahlkriterien auf Daten und Kovarianzmatrix
    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :]
    thecov = thecov[:, select]
    
    return z, dm, thecov

# Funktion zur Berechnung der Leuchtkraftentfernung im Lambda-CDM-Modell
def luminosity_distance(z, omegam, h0):
    """
    Berechnet die Leuchtkraftentfernung basierend auf Omega_M und H_0.
    """
    c = 299792.458  # Lichtgeschwindigkeit in km/s
    
    def e(z):
        return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam))
    
    integral = np.array([np.trapz(e(np.linspace(0, zi, 100)), np.linspace(0, zi, 100)) for zi in z])
    
    d_L = (c / h0) * (1 + z) * integral  # Leuchtkraftentfernung in Mpc
    
    return d_L

# Modellfunktion für das Distanzmodul
def model_func(z, omegam, h0):
    d_L = luminosity_distance(np.array([z]), omegam, h0)[0]  # Leuchtkraftentfernung für ein z
    return 5 * np.log10(d_L) + 25

# Funktion zur Berechnung von Chi²
def chi_squared(params, z, dm, cov_matrix):
    """
    Berechnet das Chi² unter Berücksichtigung der Kovarianzmatrix.
    """
    omegam, h0 = params  # Extrahiere Omega_M und H_0
    mag_model = np.array([model_func(zi, omegam, h0) for zi in z])  # Modellwerte für alle z
    delta_m = dm - mag_model  # Residuen

    inv_cov_matrix = np.linalg.inv(cov_matrix)  # Inverse der Kovarianzmatrix
    chi2 = np.dot(delta_m.T, np.dot(inv_cov_matrix, delta_m))  # delta_m^T * inv(C) * delta_m
    
    return chi2

# Funktion zur Durchführung des Fits
def fit_omega_m_h0(z, dm, cov_matrix):
    """
    Fit für Omega_M und H_0 unter Verwendung des Pantheon+SHOES-Datensatzes.
    """
    initial_guess = [0.3, 70]  # Startwerte für Omega_M und H_0

    result = minimize(
        lambda params: chi_squared(params, z, dm, cov_matrix),
        x0=initial_guess,
        bounds=[(0.1, 1.0), (50.0, 90.0)]  # Grenzen für Omega_M und H_0
    )
    
    return result.x

# Pfade zu den Dateien
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"

# Daten einlesen
z_pantheon, dm_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)

# Fit durchführen
omega_m_fit, h_0_fit = fit_omega_m_h0(z_pantheon, dm_pantheon, cov_matrix_pantheon)

# Ergebnisse ausgeben
print(f"Beste Werte: Omega_M={omega_m_fit:.3f}, H_0={h_0_fit:.3f}")

# Fehler aus der Kovarianzmatrix berechnen (Diagonalelemente)
dm_errors = np.sqrt(np.diag(cov_matrix_pantheon))

# Plot erstellen
plt.figure(figsize=(10, 6))
plt.errorbar(z_pantheon,
             dm_pantheon,
             yerr=dm_errors,
             fmt='o',
             color='red',
             label="Pantheon+SHOES Data")

# Fitkurve plotten
z_fit_values = np.linspace(min(z_pantheon), max(z_pantheon), 500)
dm_fit_values = [model_func(zf, omega_m_fit, h_0_fit) for zf in z_fit_values]
plt.plot(z_fit_values,
         dm_fit_values,
         label=f"Fit ($\Omega_m$={omega_m_fit:.3f}, $H_0$={h_0_fit:.3f})",
         color='blue')

plt.xlabel("Redshift $z$")
plt.ylabel("Relative Magnitude $m_{rel}$")
plt.title(r"Pantheon+SHOES: Relative Magnitude vs. Redshift Fit ($\Lambda$CDM Model)")
plt.savefig('m_rel_vs_z_fit_CDM', dpi=300)
plt.legend()
plt.grid()
plt.show()

### Contour Plot for Uncertainies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

# --- Deine eigene Leuchtkraftentfernung im Lambda-CDM-Modell ---
def luminosity_distance(z, omegam, h0):
    c = 299792.458  # km/s

    def e(z_):
        return 1.0 / np.sqrt(omegam * (1 + z_)**3 + (1 - omegam))
    
    z = np.array(z)
    integral = np.array([
        np.trapz(e(np.linspace(0, zi, 200)), np.linspace(0, zi, 200)) if zi > 0 else 0
        for zi in z
    ])
    d_L = (c / h0) * (1 + z) * integral
    return d_L  # in Mpc

# --- Modellfunktion für das Distanzmodul ---
def model_func(z, omegam, h0):
    d_L = luminosity_distance(z, omegam, h0)
    return 5 * np.log10(d_L) + 25

# --- Chi²-Funktion ---
def chi_squared(params, z, dm, inv_cov_matrix):
    omegam, h0 = params
    mag_model = model_func(z, omegam, h0)
    delta_m = dm - mag_model
    return delta_m @ inv_cov_matrix @ delta_m

# --- Pantheon+SH0ES-Daten laden ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, sep=' ')
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = 1701
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    select = z > 0.01
    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :][:, select]

    return z, dm, thecov

# --- Pfade ---
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"
chi2_file = "chi2_grid_custom.npy"
like_file = "likelihood_grid_custom.npy"

# --- Parameterbereich ---
omega_m_vals = np.linspace(0.1, 0.8, 170)
h_0_vals = np.linspace(60, 90, 170)
H_0_mesh, Omega_m_mesh = np.meshgrid(h_0_vals, omega_m_vals)

# --- Daten ---
z_pantheon, dm_pantheon, cov_matrix = load_pantheon_sh0es_data(data_file, cov_file)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# --- Chi²-Gitter berechnen (oder laden) ---
try:
    chi2_grid = np.load(chi2_file)
    likelihood_grid = np.load(like_file)
    print("Gespeicherte Grids geladen.")
except FileNotFoundError:
    chi2_grid = np.zeros((len(omega_m_vals), len(h_0_vals)))
    likelihood_grid = np.zeros_like(chi2_grid)

    with tqdm(total=len(omega_m_vals) * len(h_0_vals), ncols=100, desc="Berechne Chi²-Gitter") as pbar:
        for i, omega_m in enumerate(omega_m_vals):
            for j, h_0 in enumerate(h_0_vals):
                chi2 = chi_squared([omega_m, h_0], z_pantheon, dm_pantheon, inv_cov_matrix)
                chi2_grid[i, j] = chi2
                likelihood_grid[i, j] = np.exp(-0.5 * chi2)
                pbar.update(1)

    np.save(chi2_file, chi2_grid)
    np.save(like_file, likelihood_grid)
    print("Chi² und Likelihood gespeichert.")
    
# --- Plot ---
min_chi2 = chi2_grid.min()
contour_levels = [min_chi2 + delta for delta in [2.3, 6.17, 11.8]]  # 1σ, 2σ, 3σ

relevant_indices = np.where((chi2_grid >= contour_levels[0]) & (chi2_grid <= contour_levels[-1]))
min_h_0 = h_0_vals[relevant_indices[1]].min()
max_h_0 = h_0_vals[relevant_indices[1]].max()
min_omega_m = omega_m_vals[relevant_indices[0]].min()
max_omega_m = omega_m_vals[relevant_indices[0]].max()

plt.figure(figsize=(10, 8))
cs = plt.contour(H_0_mesh,
                 Omega_m_mesh,
                 chi2_grid,
                 levels=contour_levels,
                 colors=['blue', 'green', 'red'])

plt.xlabel('$H_0$ [km/s/Mpc]')
plt.ylabel('$\Omega_m$')
plt.title(r'$\chi^2$ Contours for $\Omega_m$ and $H_0$')


handles = [plt.Line2D([0], [0], color=color) for color in ['blue', 'green', 'red']]
labels = ['1σ (Δχ²=2.3)', '2σ (Δχ²=6.17)', '3σ (Δχ²=11.8)']

plt.xlim(min_h_0 - 1, max_h_0 + 1)
plt.ylim(min_omega_m - 0.01, max_omega_m + 0.01)
plt.tight_layout()
plt.savefig('contourplot_CDM_thesis.png', dpi=300)
plt.show()

# Index des Minimums im Gitter
min_index = np.unravel_index(np.argmin(chi2_grid), chi2_grid.shape)
best_omega_m = omega_m_vals [min_index[0]]
best_h_0 = h_0_vals[min_index[1]]

# Ergebnisse ausgeben
print (f"Beste Werte:")
print (f"Omega_m = {best_omega_m: .4f}")
print (f"H_0 = {best_h_0: .4f}")

### MCMC Contour Plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from scipy.integrate import quad
import pandas as pd
import emcee
from matplotlib.lines import Line2D

# Konstante für Lichtgeschwindigkeit
c = 299792.458  # km/s

# Modellfunktion: Distanzmodul aus H0 und Omega_m
def e(z, omegam):
    return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam))

def luminosity_distance(z, omegam, h0):
    integral, _ = quad(e, 0, z, args=(omegam,))
    d_L = (c / h0) * (1 + z) * integral  # in Mpc
    return d_L

def model_func(z, omegam, h0):
    d_L = luminosity_distance(z, omegam, h0)
    return 5 * np.log10(d_L) + 25  # Distance modulus

# Log-Likelihood
def log_likelihood(params, z, dm, cov_matrix):
    omegam, h0 = params
    mag_model = np.array([model_func(zi, omegam, h0) for zi in z])
    delta_m = dm - mag_model
    inv_cov = np.linalg.inv(cov_matrix)
    chi2 = delta_m.T @ inv_cov @ delta_m
    return -0.5 * chi2, chi2  # Return both log-likelihood and chi2

# Log-Prior
def log_prior(params):
    omegam, h0 = params
    if 0.1 < omegam < 0.999 and 50 < h0 < 90:
        return 0.0
    return -np.inf

# Log-Posterior
def log_posterior(params, z, dm, cov_matrix):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf, np.inf  # Return inf for chi2 if prior is invalid
    ll, chi2 = log_likelihood(params, z, dm, cov_matrix)
    return lp + ll, chi2

# --- Daten einlesen ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, delim_whitespace=True)
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = len(dat)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    select = z > 0.01
    z = z[select]
    dm = dm[select]
    thecov = cov1[select][:, select]
    return z, dm, thecov

# --- Dateipfade ---
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"
# --- Daten laden ---
z_pantheon, mu_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)

# --- emcee ---
ndim = 2  # Anzahl der Parameter: Omega_m, H_0
nwalkers = 30
nsteps = 5000
initial_guess = [0.3, 73.0]
pos = initial_guess + 1e-1 * np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(z_pantheon, mu_pantheon, cov_matrix_pantheon))
sampler.run_mcmc(pos, nsteps, progress=True)
samples = sampler.get_chain(discard=1000, flat=True)
Omega_m_samples, H_0_samples = samples[:, 0], samples[:, 1]

# Visualisierung der Startpositionen der Walker für Omega_m und H_0
plt.figure(figsize=(10, 6))
plt.scatter(pos[:, 0], pos[:, 1], label='Walker Startpositionen')  # Omega_m vs. H_0
plt.xlabel(r'$\Omega_m$', fontsize=14)
plt.ylabel(r'$H_0$', fontsize=14)
plt.title('Startpositionen der Walker (Omega_m vs. H_0)', fontsize=16)
plt.grid(True)
plt.show()

# --- Beste Parameter (Maximum Likelihood) finden ---
log_likelihoods = np.array([log_likelihood(params, z_pantheon, mu_pantheon, cov_matrix_pantheon) for params in samples])
best_index = np.argmax(log_likelihoods)
best_params = samples[best_index]
best_Omega_m, best_H_0 = best_params
chi2_min = -2 * log_likelihoods[best_index]


# Dynamische Achsenbereiche (mit optionaler Ausreißer-Filterung)
# Verwende entweder:
h0_range = (np.min(H_0_samples) - 1, np.max(H_0_samples) + 1)
omegam_range = (np.min(Omega_m_samples) - 0.01, np.max(Omega_m_samples) + 0.01)

# Oder robust gegen Ausreißer:
# h0_range = np.percentile(H_0_samples, [0.5, 99.5])
# omegam_range = np.percentile(Omega_m_samples, [0.5, 99.5])

# 2D Histogramm
nbins = 70
H, xedges, yedges = np.histogram2d(H_0_samples, Omega_m_samples, bins=nbins,
                                   range=[h0_range, omegam_range], density=True)
H_smooth = gaussian_filter(H, sigma=1.0)

# Achsen-Mitten für das Meshgrid
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
X, Y = np.meshgrid(xbin, ybin)

# Glaubwürdigkeitsniveaus
sorted_H = np.sort(H_smooth.flatten())[::-1]
cumulative = np.cumsum(sorted_H)
cumulative /= cumulative[-1]
cred_levels = [0.683, 0.954, 0.997]
contour_levels = [sorted_H[np.searchsorted(cumulative, level)] for level in cred_levels]
contour_levels = np.sort(contour_levels)

# Plot
plt.figure(figsize=(10, 8))
plt.contourf(X, Y, H_smooth.T, levels=50, cmap='Blues', alpha=0.7)

# Konturen zeichnen
contour_colors = ['red', 'green', 'blue']
labels = [r'99.7% credible region (3$\sigma$)',
          r'95.4% credible region (2$\sigma$)',
          r'68.3% credible region (1$\sigma$)']
lines = []

for level, color in zip(contour_levels, contour_colors):
    line = plt.contour(X, Y, H_smooth.T, levels=[level], colors=[color], linewidths=2)
    lines.append(line.legend_elements()[0][0])

# Best-Fit Punkt (ersetze diese Werte ggf. mit echten Fits)
best_Omega_m = np.mean(Omega_m_samples)
best_H_0 = np.mean(H_0_samples)

best_label = fr'Best Fit: $\Omega_m = {best_Omega_m:.3f}$, $H_0 = {best_H_0:.3f}$'
dummy_line = Line2D([0], [0], color='white', alpha=0)
lines.append(dummy_line)
labels.append(best_label)

# Legende, Achsen, Titel
plt.legend(lines, labels, loc='upper right', fontsize=12)
plt.title(r'Contour Plots of $\Omega_m$ vs $H_0$ (5000 Steps Total)', fontsize=18)
plt.xlabel(r'$H_0$', fontsize=14)
plt.ylabel(r'$\Omega_m$', fontsize=14)
plt.xlim(h0_range)
plt.ylim(omegam_range)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("contour_plot_emcee_thesis_CDM.png", dpi=300)
plt.show()

np.savez("dgp_mcmc_results_emcee_CDM.npz",
         Omega_m_samples=Omega_m_samples,
         H_0_samples=H_0_samples,
         best_params=best_params,
         chi2_min=chi2_min,
         walker_positions=np.array(pos),
         chain=sampler.get_chain(),  # for emcee >= 3
         lnprobability=sampler.get_log_prob())  # for emcee >= 3

### Histograms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm  # Für Fortschrittsbalken

# --------------------------------------------
# Daten einlesen & vorbereiten
# --------------------------------------------

# Lade Pantheon+SH0ES-Daten
dat = pd.read_csv('/Users/insertdatapath/Pantheon+SH0ES.dat', sep=' ')
z = np.array(dat['zHD'])
dm = np.array(dat['MU_SH0ES'])

# Lade Kovarianzmatrix
ncol = 1701
cov1 = np.loadtxt("/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov")
cov1.shape = (ncol, ncol)
thecov = cov1

# Filtere lokale Supernovae (nur z > 0.01)
select = z > 0.01
z_filtered = z[select]
dm_filtered = dm[select]
thecov_filtered = thecov[select, :][:, select]

# Für später
z = z_filtered
dm = dm_filtered
cov_matrix = thecov_filtered

# --------------------------------------------
# Kosmologie-Modellfunktionen
# --------------------------------------------

def luminosity_distance(z, omegam, h0):
    c = 299792.458  # Lichtgeschwindigkeit in km/s
    def e(z):
        return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam))
    integral = np.array([np.trapz(e(np.linspace(0, zi, 100)), np.linspace(0, zi, 100)) for zi in z])
    d_L = (c / h0) * (1 + z) * integral
    return d_L

def model_func(z, omegam, h0):
    d_L = luminosity_distance(np.array([z]), omegam, h0)[0]
    return 5 * np.log10(d_L) + 25

# --------------------------------------------
# Chi²-Minimierung
# --------------------------------------------

def chi_squared(params, z, dm, cov_matrix):
    omegam, h0 = params
    mag_model = np.array([model_func(zi, omegam, h0) for zi in z])
    delta_m = dm - mag_model
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    chi2 = np.dot(delta_m.T, np.dot(inv_cov_matrix, delta_m))
    return chi2

# --------------------------------------------
# Monte-Carlo-Simulation
# --------------------------------------------

def simulate_supernovae(z, omegam, h0, cov_matrix):
    dm_model = np.array([model_func(zi, omegam, h0) for zi in z])
    error = np.random.multivariate_normal(np.zeros(len(z)), cov_matrix)
    dm_sim = dm_model + error
    return dm_sim, cov_matrix

# Spezifische Werte für Omega_M und H_0
omegam = 0.331
h0 = 73.244
n_simulations = 1500

# Ergebnisse speichern
fitted_omegam = []
fitted_h0 = []

# Simulation
print(f"Starte Monte Carlo Simulation für Omega_M={omegam:.3f}, H0={h0:.2f}...")

for _ in tqdm(range(n_simulations)):
    dm_sim, cov_sim = simulate_supernovae(z, omegam, h0, cov_matrix)

    result = minimize(chi_squared, x0=[0.3, 70], args=(z, dm_sim, cov_sim),
                      bounds=[(0.1, 0.5), (60, 80)], method='L-BFGS-B')

    if result.success:
        omega_fit, h0_fit = result.x
        fitted_omegam.append(omega_fit)
        fitted_h0.append(h0_fit)

# --------------------------------------------
# Ergebnisse visualisieren
# --------------------------------------------

fitted_omegam = np.array(fitted_omegam)
fitted_h0 = np.array(fitted_h0)

plt.figure(figsize=(12, 6))

# Omega_M Histogramm
plt.subplot(1, 2, 1)
plt.hist(fitted_omegam, bins=30, color='skyblue', edgecolor='black')
plt.axvline(np.mean(fitted_omegam), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_omegam):.3f}")
plt.xlabel(r'Fitted $\Omega_m$')
plt.ylabel('Incidence')
plt.title(r'Distribution of $\Omega_m$')
plt.legend()

# H_0 Histogramm
plt.subplot(1, 2, 2)
plt.hist(fitted_h0, bins=30, color='lightgreen', edgecolor='black')
plt.axvline(np.mean(fitted_h0), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_h0):.2f}")
plt.xlabel(r'Fitted $H_0$')
plt.ylabel('Incidence')
plt.title('Distribution of $H_0$')
plt.legend()

plt.tight_layout()
plt.show()

# Daten speichern
np.savez("montecarlo_histo_CDM_results.npz",
         fitted_omegam=fitted_omegam,
         fitted_h0=fitted_h0,
         z=z,
         dm=dm,
         cov_matrix=cov_matrix,
         omegam=omegam,
         h0=h0,
         n_simulations=n_simulations)
print("Simulationsergebnisse gespeichert in 'montecarlo_histo_CDM_results.npz'")